In [ ]:
import pandas as pd
import numpy as np
import jsonlines
import json
import cv2
import os
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.signal import savgol_filter
from scipy.stats import mode
from nba_api.stats.static import teams
from nba_api.stats.endpoints import commonteamroster

In [ ]:
def get_team_id(team_abbrev):
  return teams.find_team_by_abbreviation(team_abbrev)['id']

In [ ]:
def get_team_player_num(team_abbrev, num):
  team_roster = commonteamroster.CommonTeamRoster(get_team_id(team_abbrev), season="2023-24")
  team_roster_df = team_roster.get_data_frames()[0]
  player = team_roster_df[team_roster_df["NUM"] == str(num)]
  if len(player) == 0:
    return None
  else:
    return list(player["PLAYER"].values)

In [ ]:
def compute_center(x1, y1, x2, y2):
  return (x1 + x2) / 2, (y1 + y2) / 2

In [ ]:
def compute_centers_for_tracks(x1_list, y1_list, x2_list, y2_list):
  x = []
  y = []
  for i in range(len(x1_list)):
    center_x, center_y = compute_center(x1_list[i], y1_list[i], x2_list[i], y2_list[i])
    x.append(center_x)
    y.append(center_y)
  return x, y

In [ ]:
def get_val_counts(list):
  data = pd.Series(list).value_counts()
  vals = []
  counts = []
  for index, value in data.items():
    vals.append(index)
    counts.append(value)

  return pd.DataFrame({"NUM": vals, "COUNT": counts})

In [ ]:
#Use a Savitzky-Golay filter to smooth out trajectories
def savitzky_golay(x, y, t, window_size=3, poly_order=2):
  smooth_x = savgol_filter(x, window_size, poly_order)
  smooth_y = savgol_filter(y, window_size, poly_order)


  plt.plot(t, x, 'o', label='Original x')
  plt.plot(t, y, 'o', label='Original y')
  plt.plot(t, smooth_x, label='Smoothed x')
  plt.plot(t, smooth_y, label='Smoothed y')
  plt.legend()
  plt.show()

  return smooth_x, smooth_y


In [ ]:
def get_track_frame_length(track_data):
  return len(track_data["FRAME_NUM"])

In [ ]:
def get_track_lengths(json_out):
  track_lengths = {}
  for i in json_out.keys():
    if i != "META":
      track_lengths[i] = get_track_frame_length(json_out[i])

  return track_lengths

In [ ]:
def largest_n_elements(in_list, n):
  sorted_list = sorted(in_list, reverse=True)
  return sorted_list[:n]

In [ ]:
def largest_n_keys(dictionary, n):
  sorted_items = sorted(dictionary.items(), key=lambda x: x[1], reverse=True)

  largest_keys = [item[0] for item in sorted_items[:n]]

  return largest_keys

In [ ]:
def get_n_longest_tracks(json_out, n):
  longest_tracks = largest_n_keys(get_track_lengths(json_out), n)

  out = {}
  for i in longest_tracks:
    out[i] = json_out[str(i)]

  return out

In [ ]:
def get_initial_frame(track):
  return track["FRAME_NUM"][0]

In [ ]:
def get_final_frame(track):
  return track["FRAME_NUM"][-1]

In [ ]:
def get_frame_range(track):
  return str(get_initial_frame(track)), str(get_final_frame(track))

In [ ]:
def get_all_initial_frames(json_out):
  return [get_initial_frame(json_out[i]) for i in json_out.keys()]

In [ ]:
def get_all_final_frames(json_out):
  return [get_final_frame(json_out[i]) for i in json_out.keys()]

In [ ]:
def get_all_track_ranges(json_out):
  range_dict = {}
  for i in json_out.keys():
    range_dict[i] = get_frame_range(json_out[i])

  return range_dict

In [ ]:
def get_all_track_info(json_out):
  out = {}

  out["TRACK_NUM"] = [i for i in json_out.keys()]
  out["i_FRAME"] = [get_initial_frame(json_out[i]) for i in json_out.keys()]
  out["f_FRAME"] = [get_final_frame(json_out[i]) for i in json_out.keys()]
  out["LENGTH"] = [get_track_frame_length(json_out[i]) for i in json_out.keys()]

  return pd.DataFrame(out)

In [ ]:
def get_video(video_path):
  video = cv2.VideoCapture(video_path)

  if not video.isOpened():
    raise IOError("Error: Could not open video file.")

  return video

In [ ]:
def get_frame(video, frame_number):
  video.set(cv2.CAP_PROP_POS_FRAMES, frame_number)

  ret, frame = video.read()

  if not ret:
    raise IOError("Error: Could not read frame.")

  return frame

In [ ]:
def load_track_file(file_path):
  with open(file_path, "r") as f:
    json_data = json.load(f)

  # Extract the title from the filename
  filename = os.path.basename(file_path)
  title = filename.split(".")[0].replace("OUT_", "")

  return json_data, title

In [ ]:
def animate_dot_plotly(x_coordinates, y_coordinates, time_coordinates):
    # Create subplot
    fig = make_subplots(rows=1, cols=1, subplot_titles=['Animated Dot Plot'])

    # Scatter plot with initial point
    scatter = go.Scatter(x=[x_coordinates[0]], y=[y_coordinates[0]], mode='markers', marker=dict(color='blue', size=10))
    fig.add_trace(scatter)

    # Set layout
    fig.update_layout(
        xaxis=dict(range=[min(x_coordinates) - 1, max(x_coordinates) + 1]),
        yaxis=dict(range=[min(y_coordinates) - 1, max(y_coordinates) + 1]),
        showlegend=False
    )

    # Update function for animation
    def update(frame):
        scatter.x = [x_coordinates[frame]]
        scatter.y = [y_coordinates[frame]]

    # Create animation frames
    frames = [go.Frame(data=[go.Scatter(x=x_coordinates[:frame + 1], y=y_coordinates[:frame + 1], mode='markers',
                                         marker=dict(color='blue', size=10))],
                       traces=[0],
                       name=f'Frame {frame}')
              for frame in range(1, len(time_coordinates))]

    fig.frames = frames

    # Add animation settings
    animation_settings = dict(frame=dict(duration=1000, redraw=True), fromcurrent=True)
    fig.update_layout(updatemenus=[dict(type='buttons', showactive=False, buttons=[dict(label='Play',
                                            method='animate', args=[None, animation_settings])])])

    # Show the plot
    fig.show()


In [ ]:
data, title = load_track_file("test_out (3).json")

In [ ]:
track_info = get_all_track_info(data)

In [ ]:
track_info

,TRACK_NUM,i_FRAME,f_FRAME,LENGTH
0,1,3,137,120
1,2,3,74,39
2,3,3,81,64
3,4,3,19,15
4,6,3,535,507
...,...,...,...,...
96,331,622,624,3
97,333,631,651,21
98,334,636,639,4
99,338,653,660,8
